# Transformación y limpieza de datos

* Homogeneizar datos, ver tipos, nulos, errores, mayusculas, etc. para luego hacer un EDA y ETL. Observo y apuntamos todo lo que hay que limpiar y transformar.


In [73]:
import pandas as pd
import numpy as np

In [74]:
df = pd.read_csv('Data/online_sales_limpiar.csv', index_col=0)
#Elimino la comuna unmamed que se genera.


In [75]:
df.sample(5)

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country,discount,payment_method,shipping_cost,category,sales_channel,return_status,shipment_provider,warehouse_location,order_priority,InvoiceNO
28548,878459,SKU_1660,NOTEBOOK,23,2023-febrero-22 03:00,"73,59",54696.0,PORTUGAL,0.19,Credit Card,"28,37",Accessories,IN-STORE,0,DHL,ROME,MEDIUM,878459
788,383821,SKU_1396,HEADPHONES,16,2020-febrero-01 09:00,"40,11",62733.0,AUSTRALIA,0.08,paypall,"18,93",Stationery,ONLINE,0,UPS,BERLIN,MEDIUM,383821
24490,286502,SKU_1783,WHITE MUG,42,2022-septiembre-11 19:00,"11,01",53481.0,SWEDEN,0.20,paypall,"18,12",Electronics,ONLINE,0,FedEx,LONDON,MEDIUM,286502
15869,353800,SKU_1262,WHITE MUG,15,2021-septiembre-29 19:00,"28,81",89048.0,SWEDEN,0.39,Credit Card,"11,42",Accessories,ONLINE,1,Royal Mail,LONDON,MEDIUM,353800
10872,993164,SKU_1867,NOTEBOOK,10,2021-marzo-12 09:00,"13,8",64546.0,FRANCE,0.28,Credit Card,"10,51",Furniture,ONLINE,1,DHL,ROME,HIGH,993164


In [76]:
df.columns

Index(['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country', 'discount', 'payment_method',
       'shipping_cost', 'category', 'sales_channel', 'return_status',
       'shipment_provider', 'warehouse_location', 'order_priority',
       'InvoiceNO'],
      dtype='object')

-- EDA previo
-- Se observan columnas que parecen repetidas y se analizan los tipo de datos erroneos, mayusculas, valores negativos, etc. para luego transformarlos.

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36340 entries, 0 to 36339
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   invoice_no          36340 non-null  int64  
 1   stock_code          36340 non-null  object 
 2   description         36340 non-null  object 
 3   quantity            36340 non-null  int64  
 4   invoice_date        36340 non-null  object 
 5   unit_price          36340 non-null  object 
 6   customer_id         32730 non-null  float64
 7   country             36340 non-null  object 
 8   discount            36340 non-null  float64
 9   payment_method      36340 non-null  object 
 10  shipping_cost       34576 non-null  object 
 11  category            36340 non-null  object 
 12  sales_channel       36340 non-null  object 
 13  return_status       36340 non-null  int64  
 14  shipment_provider   36340 non-null  object 
 15  warehouse_location  33820 non-null  object 
 16  order_pri

In [78]:
df.isnull().sum()

invoice_no               0
stock_code               0
description              0
quantity                 0
invoice_date             0
unit_price               0
customer_id           3610
country                  0
discount                 0
payment_method           0
shipping_cost         1764
category                 0
sales_channel            0
return_status            0
shipment_provider        0
warehouse_location    2520
order_priority           0
InvoiceNO                0
dtype: int64

In [79]:
#df.describe()
df.describe().T

,count,mean,std,min,25%,50%,75%,max
invoice_no,36340.0,551662.412631,260560.241463,100005.0,326151.25,554002.00,777560.75,999997.000000
quantity,36340.0,22.420914,17.816181,-50.0,11.00,23.00,37.00,49.000000
customer_id,32730.0,55014.170975,25957.694626,10001.0,32688.50,55245.50,77448.50,99998.000000
discount,36340.0,0.275637,0.227978,0.0,0.13,0.26,0.38,1.998946
return_status,36340.0,0.098597,0.298124,0.0,0.00,0.00,0.00,1.000000
InvoiceNO,36340.0,551662.412631,260560.241463,100005.0,326151.25,554002.00,777560.75,999997.000000


In [80]:
df.describe(include='O').T

,count,unique,top,freq
stock_code,36340,1000,SKU_1724,57
description,36340,11,WALL CLOCK,3378
invoice_date,36340,35064,2021-enero-16 08:00,3
unit_price,36340,10540,"47,54",12
country,36340,12,FRANCE,3123
payment_method,36340,3,Bank Transfer,12325
shipping_cost,34576,2501,"5,34",30
category,36340,5,Accessories,7323
sales_channel,36340,2,ONLINE,18274
shipment_provider,36340,4,UPS,9113


In [81]:
#Value_counts devuelve fecuencia de aparicion de cada una de las categorias de la columna seleccionada
df['description'].value_counts()

description
WALL CLOCK        3378
HEADPHONES        3357
OFFICE CHAIR      3354
WHITE MUG         3351
USB CABLE         3332
BACKPACK          3307
BLUE PEN          3286
DESK LAMP         3274
NOTEBOOK          3262
WIRELESS MOUSE    3220
T-SHIRT           3219
Name: count, dtype: int64

In [82]:
#Columnas categóricas df.select_dtypes(include='O').columns
#Iterando tengo el value_counts de cada una de las columnas categóricas para poder estudiarlas y ver todo lo que hay por limpiar.
for col in df.select_dtypes(include='O').columns:
    print(df[col].value_counts())
    print('----------------------------------')

stock_code
SKU_1724    57
SKU_1044    55
SKU_1743    55
SKU_1953    55
SKU_1761    54
            ..
SKU_1212    20
SKU_1273    20
SKU_1126    19
SKU_1122    19
SKU_1031    19
Name: count, Length: 1000, dtype: int64
----------------------------------
description
WALL CLOCK        3378
HEADPHONES        3357
OFFICE CHAIR      3354
WHITE MUG         3351
USB CABLE         3332
BACKPACK          3307
BLUE PEN          3286
DESK LAMP         3274
NOTEBOOK          3262
WIRELESS MOUSE    3220
T-SHIRT           3219
Name: count, dtype: int64
----------------------------------
invoice_date
2021-enero-16 08:00        3
2020-junio-19 14:00        3
2023-febrero-16 04:00      3
2022-noviembre-07 23:00    3
2022-junio-01 23:00        3
                          ..
2021-mayo-11 03:00         1
2021-mayo-11 02:00         1
2021-mayo-11 01:00         1
2021-mayo-11 00:00         1
2023-diciembre-31 23:00    1
Name: count, Length: 35064, dtype: int64
----------------------------------
unit_price
47,5

------
# LIMPIEZA

## 1. Eliminar columnas redundantes



In [83]:
df.columns

Index(['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country', 'discount', 'payment_method',
       'shipping_cost', 'category', 'sales_channel', 'return_status',
       'shipment_provider', 'warehouse_location', 'order_priority',
       'InvoiceNO'],
      dtype='object')

In [84]:
#Analizo columna duplicada
df[['invoice_no', 'InvoiceNO']]

,invoice_no,InvoiceNO
0,221958,221958
1,771155,771155
2,231932,231932
3,465838,465838
4,359178,359178
...,...,...
36335,777548,777548
36336,166019,166019
36337,624739,624739
36338,950706,950706


In [85]:
#Metodo equals comrpueba si las dos columnas son iguales.
df['invoice_no'].equals(df['InvoiceNO'])


True

In [86]:
#Elimino la columna duplicada InvoiceNO
df.drop(columns=['InvoiceNO'], axis=1, inplace=True)
#Chequeo que se elimino

In [87]:
df

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country,discount,payment_method,shipping_cost,category,sales_channel,return_status,shipment_provider,warehouse_location,order_priority
0,221958,SKU_1964,WHITE MUG,38,2020-enero-01 00:00,"1,71",37039.0,AUSTRALIA,0.470000,Bank Transfer,"10,79",Apparel,IN-STORE,0,UPS,LONDON,MEDIUM
1,771155,SKU_1241,WHITE MUG,18,2020-enero-01 01:00,"41,25",19144.0,SPAIN,0.190000,paypall,"9,51",Electronics,ONLINE,0,UPS,ROME,MEDIUM
2,231932,SKU_1501,HEADPHONES,49,2020-enero-01 02:00,"29,11",50472.0,GERMANY,0.350000,Bank Transfer,"23,03",Electronics,ONLINE,1,UPS,BERLIN,HIGH
3,465838,SKU_1760,DESK LAMP,14,2020-enero-01 03:00,"76,68",96586.0,NETHERLANDS,0.140000,paypall,"11,08",Accessories,ONLINE,0,Royal Mail,ROME,LOW
4,359178,SKU_1386,USB CABLE,-30,2020-enero-01 04:00,"-68,11",NaN,UNITED KINGDOM,1.501433,Bank Transfer,NaN,Electronics,IN-STORE,0,FedEx,NaN,MEDIUM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36335,777548,SKU_1210,HEADPHONES,6,2023-diciembre-31 19:00,"92,64",36062.0,AUSTRALIA,0.140000,paypall,"8,33",Apparel,IN-STORE,0,Royal Mail,AMSTERDAM,HIGH
36336,166019,SKU_1797,OFFICE CHAIR,21,2023-diciembre-31 20:00,"36,19",99978.0,NORWAY,0.430000,paypall,"10,82",Electronics,ONLINE,1,DHL,PARIS,HIGH
36337,624739,SKU_1835,WIRELESS MOUSE,10,2023-diciembre-31 21:00,"49,16",83182.0,NORWAY,0.120000,paypall,"17,09",Accessories,IN-STORE,0,FedEx,AMSTERDAM,HIGH
36338,950706,SKU_1326,OFFICE CHAIR,7,2023-diciembre-31 22:00,"42,31",28358.0,NORWAY,0.220000,Bank Transfer,"15,5",Stationery,IN-STORE,1,DHL,AMSTERDAM,LOW


## 2. Homogeneizar el nombre de las columas.

In [88]:
df.columns

Index(['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country', 'discount', 'payment_method',
       'shipping_cost', 'category', 'sales_channel', 'return_status',
       'shipment_provider', 'warehouse_location', 'order_priority'],
      dtype='object')

In [89]:
#Reemplazo al comienzo palabra minusculas por mayusculas en columna 0
#columna = df.columns[0]
#columna.replace('_', ' ').title().replace(' ', '_')

columna = df.columns[0]
columna.title()

'Invoice_No'

In [ ]:
#Diccionario con Bulcle para cambiar el nombre de las columnas

'''nombres_columnas = {}

for col in df.columns:
    nombres_columnas[col] = col.title()'''

In [93]:
#Optimizo el diccionario con comprensión de listas
nombres_columnas = {col: col.title() for col in df.columns}

In [ ]:
#Resultado
nombres_columnas

{'invoice_no': 'Invoice_No',
 'stock_code': 'Stock_Code',
 'description': 'Description',
 'quantity': 'Quantity',
 'invoice_date': 'Invoice_Date',
 'unit_price': 'Unit_Price',
 'customer_id': 'Customer_Id',
 'country': 'Country',
 'discount': 'Discount',
 'payment_method': 'Payment_Method',
 'shipping_cost': 'Shipping_Cost',
 'category': 'Category',
 'sales_channel': 'Sales_Channel',
 'return_status': 'Return_Status',
 'shipment_provider': 'Shipment_Provider',
 'warehouse_location': 'Warehouse_Location',
 'order_priority': 'Order_Priority'}

In [95]:
#Chequeo que se renombraron las columnas
df.rename(columns=nombres_columnas, inplace=True)

In [96]:
df.columns

Index(['Invoice_No', 'Stock_Code', 'Description', 'Quantity', 'Invoice_Date',
       'Unit_Price', 'Customer_Id', 'Country', 'Discount', 'Payment_Method',
       'Shipping_Cost', 'Category', 'Sales_Channel', 'Return_Status',
       'Shipment_Provider', 'Warehouse_Location', 'Order_Priority'],
      dtype='object')

## 3. Homogeneizar datos categóricos.

Ponemos en minúscula

In [152]:
#Ponemos en minúscula el contenido de las columnas categoricas, title() primer letra mayuscula.
for col in df.select_dtypes(include='O').columns:
    df[col] = df[col].str.lower()



In [ ]:
#Chequeo
df.sample(5)

,Invoice_No,Stock_Code,Description,Quantity,Invoice_Date,Unit_Price,Customer_Id,Country,Discount,Payment_Method,Shipping_Cost,Category,Sales_Channel,Return_Status,Shipment_Provider,Warehouse_Location,Order_Priority
11490,994285,sku_1532,office chair,29,2021-abril-06 09:00,"95,68",46035.0,italy,0.17,credit card,"24,71",furniture,online,1,fedex,paris,medium
36253,567353,sku_1072,wireless mouse,8,2023-diciembre-28 14:00,"66,13",72627.0,belgium,0.34,paypall,"29,16",accessories,in-store,0,dhl,paris,high
4831,438384,sku_1577,office chair,48,2020-julio-12 09:00,"68,32",32145.0,belgium,0.06,paypall,"19,09",stationery,online,0,royal mail,london,low
25899,384933,sku_1318,backpack,39,2022-noviembre-07 08:00,"11,62",24682.0,belgium,0.12,paypall,"7,84",furniture,online,1,royal mail,paris,medium
35132,951539,sku_1275,backpack,18,2023-noviembre-13 03:00,"60,98",17672.0,france,0.14,credit card,"24,65",stationery,in-store,0,fedex,london,medium


## 4.Cambio tipo de datos.

In [ ]:
#1- Los meses estan en texto, generamos diccionario de meses para convertir a formato numérico

meses = {'enero':'01', 
         'febrero':'02',
         'marzo':'03',
         'abril':'04',
         'mayo':'05',
         'junio':'06',
         'julio':'07',
         'agosto':'08',
         'septiembre':'09',
         'octubre':'10',
         'noviembre':'11',
         'diciembre':'12'}

In [123]:
#Reemplazo los meses en la columna Invoice_Date, uno regex para buscar patrones de texto.
df.replace({'Invoice_Date': meses}, regex=True, inplace=True)


In [124]:
df.sample(5)

,Invoice_No,Stock_Code,Description,Quantity,Invoice_Date,Unit_Price,Customer_Id,Country,Discount,Payment_Method,Shipping_Cost,Category,Sales_Channel,Return_Status,Shipment_Provider,Warehouse_Location,Order_Priority
34889,711524,sku_1578,office chair,42,2023-11-03 14:00:00,"6,9",81173.0,germany,0.450000,bank transfer,"11,72",furniture,online,0,fedex,london,low
33647,929017,sku_1596,white mug,27,2023-09-14 21:00:00,"56,43",60714.0,netherlands,0.260000,bank transfer,"26,67",stationery,online,0,dhl,berlin,medium
31696,675512,sku_1791,office chair,19,2023-06-28 19:00:00,"39,07",13997.0,norway,0.460000,bank transfer,"28,31",stationery,online,0,royal mail,paris,low
15706,914892,sku_1970,white mug,34,2021-09-23 06:00:00,"36,73",50992.0,italy,0.110000,paypall,"9,71",apparel,online,0,fedex,berlin,high
32159,902204,sku_1997,blue pen,-34,2023-07-17 11:00:00,"-78,71",NaN,france,1.589915,bank transfer,NaN,stationery,online,0,royal mail,NaN,low


In [125]:
#Invoice_Date a formato fecha
df['Invoice_Date'] = pd.to_datetime(df['Invoice_Date'], format='%Y-%m-%d %H:%M')

In [126]:
df.sample(5)

,Invoice_No,Stock_Code,Description,Quantity,Invoice_Date,Unit_Price,Customer_Id,Country,Discount,Payment_Method,Shipping_Cost,Category,Sales_Channel,Return_Status,Shipment_Provider,Warehouse_Location,Order_Priority
31174,509421,sku_1137,wireless mouse,47,2023-06-08 01:00:00,"81,88",59056.0,sweden,0.31,bank transfer,"9,99",furniture,in-store,0,dhl,rome,low
18765,500404,sku_1470,white mug,47,2022-01-24 09:00:00,"95,19",54921.0,united kingdom,0.04,bank transfer,"19,05",accessories,in-store,0,ups,london,high
7222,452215,sku_1548,usb cable,27,2020-10-16 14:00:00,"81,74",67002.0,united states,0.48,bank transfer,"5,53",apparel,in-store,1,fedex,rome,medium
5317,911050,sku_1851,desk lamp,8,2020-07-31 23:00:00,"45,6",27953.0,france,0.43,credit card,"14,02",accessories,in-store,0,fedex,rome,low
5741,979854,sku_1735,backpack,37,2020-08-18 04:00:00,"93,24",22467.0,belgium,0.01,credit card,"20,56",furniture,online,0,ups,rome,medium


In [ ]:
#Chequeo que se transformo a fecha
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36340 entries, 0 to 36339
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Invoice_No          36340 non-null  int64         
 1   Stock_Code          36340 non-null  object        
 2   Description         36340 non-null  object        
 3   Quantity            36340 non-null  int64         
 4   Invoice_Date        36340 non-null  datetime64[ns]
 5   Unit_Price          36340 non-null  object        
 6   Customer_Id         32730 non-null  float64       
 7   Country             36340 non-null  object        
 8   Discount            36340 non-null  float64       
 9   Payment_Method      36340 non-null  object        
 10  Shipping_Cost       34576 non-null  object        
 11  Category            36340 non-null  object        
 12  Sales_Channel       36340 non-null  object        
 13  Return_Status       36340 non-null  int64         


In [ ]:
#2- Cambiar a float la columna Unit_Price, Shipping_Cost, van con punto no coma.

str_float = ['Unit_Price', 'Shipping_Cost']
for col in str_float:
    df[col] = df[col].str.replace(',', '.')
    df[col] = df[col].astype(float)
    #similar a lo anterior, df[col] = df[col].apply(lambda x: float(x))



AttributeError: Can only use .str accessor with string values!

In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36340 entries, 0 to 36339
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Invoice_No          36340 non-null  int64         
 1   Stock_Code          36340 non-null  object        
 2   Description         36340 non-null  object        
 3   Quantity            36340 non-null  int64         
 4   Invoice_Date        36340 non-null  datetime64[ns]
 5   Unit_Price          36340 non-null  float64       
 6   Customer_Id         32730 non-null  float64       
 7   Country             36340 non-null  object        
 8   Discount            36340 non-null  float64       
 9   Payment_Method      36340 non-null  object        
 10  Shipping_Cost       34576 non-null  float64       
 11  Category            36340 non-null  object        
 12  Sales_Channel       36340 non-null  object        
 13  Return_Status       36340 non-null  int64         


In [136]:
df.head(5)

,Invoice_No,Stock_Code,Description,Quantity,Invoice_Date,Unit_Price,Customer_Id,Country,Discount,Payment_Method,Shipping_Cost,Category,Sales_Channel,Return_Status,Shipment_Provider,Warehouse_Location,Order_Priority
0,221958,sku_1964,white mug,38,2020-01-01 00:00:00,1.71,37039.0,australia,0.470000,bank transfer,10.79,apparel,in-store,0,ups,london,medium
1,771155,sku_1241,white mug,18,2020-01-01 01:00:00,41.25,19144.0,spain,0.190000,paypall,9.51,electronics,online,0,ups,rome,medium
2,231932,sku_1501,headphones,49,2020-01-01 02:00:00,29.11,50472.0,germany,0.350000,bank transfer,23.03,electronics,online,1,ups,berlin,high
3,465838,sku_1760,desk lamp,14,2020-01-01 03:00:00,76.68,96586.0,netherlands,0.140000,paypall,11.08,accessories,online,0,royal mail,rome,low
4,359178,sku_1386,usb cable,-30,2020-01-01 04:00:00,-68.11,NaN,united kingdom,1.501433,bank transfer,NaN,electronics,in-store,0,fedex,NaN,medium


In [137]:
#3- Return_Status reemplazo numeros 0 y 1 por texto. Genero diccionario de mapeo y aplico metodo map

diccionario_mapeo = {1: 'returned', 0: 'not returned'}

df['Return_Status'] = df['Return_Status'].map(diccionario_mapeo)
#Chequeo
df.head()

,Invoice_No,Stock_Code,Description,Quantity,Invoice_Date,Unit_Price,Customer_Id,Country,Discount,Payment_Method,Shipping_Cost,Category,Sales_Channel,Return_Status,Shipment_Provider,Warehouse_Location,Order_Priority
0,221958,sku_1964,white mug,38,2020-01-01 00:00:00,1.71,37039.0,australia,0.470000,bank transfer,10.79,apparel,in-store,not returned,ups,london,medium
1,771155,sku_1241,white mug,18,2020-01-01 01:00:00,41.25,19144.0,spain,0.190000,paypall,9.51,electronics,online,not returned,ups,rome,medium
2,231932,sku_1501,headphones,49,2020-01-01 02:00:00,29.11,50472.0,germany,0.350000,bank transfer,23.03,electronics,online,returned,ups,berlin,high
3,465838,sku_1760,desk lamp,14,2020-01-01 03:00:00,76.68,96586.0,netherlands,0.140000,paypall,11.08,accessories,online,not returned,royal mail,rome,low
4,359178,sku_1386,usb cable,-30,2020-01-01 04:00:00,-68.11,NaN,united kingdom,1.501433,bank transfer,NaN,electronics,in-store,not returned,fedex,NaN,medium


In [ ]:
#4- Saco columna con el año de la fecha
df['Year'] = df['Invoice_Date'].dt.year

In [ ]:
#5- Saco columna con el mes de la fecha
df['Month'] = df['Invoice_Date'].dt.month

In [144]:
#6- Saco columna con el día de la semana
df['Day_Week'] = df['Invoice_Date'].dt.day_name()


In [149]:
#7- Saco columna con la semana del año
df['Week'] = df['Invoice_Date'].dt.isocalendar().week

In [ ]:
#8- Saco columna con el cuatrimestre del año
df['Quarter'] = df['Invoice_Date'].dt.quarter

In [153]:
df.sample(15)

,Invoice_No,Stock_Code,Description,Quantity,Invoice_Date,Unit_Price,Customer_Id,Country,Discount,Payment_Method,...,Sales_Channel,Return_Status,Shipment_Provider,Warehouse_Location,Order_Priority,Year,Month,Day_Week,Week,Quarter
26788,499104,sku_1612,backpack,20,2022-12-12 22:00:00,38.45,74634.0,portugal,0.41000,bank transfer,...,in-store,not returned,dhl,rome,high,2022,12,monday,50,4
5747,263187,sku_1876,usb cable,30,2020-08-18 10:00:00,3.04,65396.0,united states,0.25000,bank transfer,...,online,not returned,fedex,berlin,high,2020,8,tuesday,34,3
9400,930547,sku_1434,notebook,30,2021-01-12 14:00:00,46.40,66205.0,sweden,0.43000,bank transfer,...,in-store,not returned,dhl,rome,medium,2021,1,tuesday,2,1
13508,661424,sku_1998,notebook,39,2021-06-26 18:00:00,28.15,28710.0,united kingdom,0.07000,paypall,...,in-store,not returned,ups,amsterdam,high,2021,6,saturday,25,2
9118,525643,sku_1222,headphones,-48,2021-01-01 06:00:00,-16.11,NaN,portugal,0.28000,credit card,...,online,not returned,dhl,NaN,low,2021,1,friday,53,1
30927,993148,sku_1046,wireless mouse,47,2023-05-29 01:00:00,52.34,46388.0,italy,0.24000,paypall,...,in-store,not returned,royal mail,amsterdam,medium,2023,5,monday,22,2
1185,637198,sku_1607,wireless mouse,38,2020-02-17 08:00:00,37.63,75974.0,united states,0.13000,bank transfer,...,online,not returned,ups,paris,medium,2020,2,monday,8,1
12803,605111,sku_1817,white mug,14,2021-05-29 06:00:00,67.66,93829.0,united kingdom,0.12000,credit card,...,online,not returned,dhl,amsterdam,low,2021,5,saturday,21,2
4846,967461,sku_1299,usb cable,37,2020-07-12 23:00:00,7.06,34993.0,spain,0.34000,credit card,...,in-store,not returned,ups,amsterdam,high,2020,7,sunday,28,3
28205,778960,sku_1709,white mug,42,2023-02-08 07:00:00,9.14,98488.0,germany,0.40000,credit card,...,online,returned,royal mail,berlin,medium,2023,2,wednesday,6,1


In [ ]:
#9- Guardamos el dataframe limpio, sin los indices
df.to_csv('Data/online_sales_limpio.csv', index=False)

In [ ]:
#Chequeo DF guardado limpio.
df_1 = pd.read_csv('Data/online_sales_limpio.csv')
df_1

,Invoice_No,Stock_Code,Description,Quantity,Invoice_Date,Unit_Price,Customer_Id,Country,Discount,Payment_Method,...,Sales_Channel,Return_Status,Shipment_Provider,Warehouse_Location,Order_Priority,Year,Month,Day_Week,Week,Quarter
0,221958,sku_1964,white mug,38,2020-01-01 00:00:00,1.71,37039.0,australia,0.470000,bank transfer,...,in-store,not returned,ups,london,medium,2020,1,wednesday,1,1
1,771155,sku_1241,white mug,18,2020-01-01 01:00:00,41.25,19144.0,spain,0.190000,paypall,...,online,not returned,ups,rome,medium,2020,1,wednesday,1,1
2,231932,sku_1501,headphones,49,2020-01-01 02:00:00,29.11,50472.0,germany,0.350000,bank transfer,...,online,returned,ups,berlin,high,2020,1,wednesday,1,1
3,465838,sku_1760,desk lamp,14,2020-01-01 03:00:00,76.68,96586.0,netherlands,0.140000,paypall,...,online,not returned,royal mail,rome,low,2020,1,wednesday,1,1
4,359178,sku_1386,usb cable,-30,2020-01-01 04:00:00,-68.11,NaN,united kingdom,1.501433,bank transfer,...,in-store,not returned,fedex,NaN,medium,2020,1,wednesday,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36335,777548,sku_1210,headphones,6,2023-12-31 19:00:00,92.64,36062.0,australia,0.140000,paypall,...,in-store,not returned,royal mail,amsterdam,high,2023,12,sunday,52,4
36336,166019,sku_1797,office chair,21,2023-12-31 20:00:00,36.19,99978.0,norway,0.430000,paypall,...,online,returned,dhl,paris,high,2023,12,sunday,52,4
36337,624739,sku_1835,wireless mouse,10,2023-12-31 21:00:00,49.16,83182.0,norway,0.120000,paypall,...,in-store,not returned,fedex,amsterdam,high,2023,12,sunday,52,4
36338,950706,sku_1326,office chair,7,2023-12-31 22:00:00,42.31,28358.0,norway,0.220000,bank transfer,...,in-store,returned,dhl,amsterdam,low,2023,12,sunday,52,4
